### 

In [23]:
import gensim
import pprint
import regex as re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import os
import string
import spacy
import datetime# For preprocessing

from spacy.lang.nl.examples import sentences 
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from collections import Counter
import json
import matplotlib.pyplot as plt
import itertools
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

import multiprocessing

from gensim.models import Word2Vec
from gensim.test.utils import datapath
from gensim.utils import tokenize
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import stem_text
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_numeric

In [24]:
#read all data
all_df = pd.read_csv('all_df_2607.csv')
no_ads = pd.read_csv('no_ads_2607.csv')
print(len(all_df),len(no_ads))

113595 61860


In [25]:
#initialize nlp function
nlp = spacy.load("nl_core_news_sm",disable=['ner', 'parser','tagger','textcat'])
raw_texts =  list(all_df['text'])

In [26]:
#preprocessing function
def preprocess(texts):
    """preprocess text. returns list of lists of cleaned lemmas"""
    stopword_list = stopwords.words('dutch')
    texts = [text.replace("\\n", "").strip() for text in texts]
    texts = [text.replace("\\", "").strip() for text in texts]
    docs = nlp.pipe(texts)
    cleaned_lemmas = [[t.lemma_ for t in doc if t.lemma_ not in stopword_list and not t.is_digit and not t.is_punct and len(t) > 3] for doc in docs]
    no_dups = list(cleaned_lemmas for cleaned_lemmas,_ in itertools.groupby(cleaned_lemmas))
    return no_dups
        


In [27]:
#train model function
def train_model(documents,krant,decade):
    model = gensim.models.Word2Vec(
        documents,
        size=150,
        window=10,
        min_count=5,
        workers=10,
        iter=10)
    model.save(f"model_{krant}_{decade}.wordvectors")

## Training model overall

In [28]:
# all_model = gensim.models.Word2Vec(
#     preprocess(list(all_df['text'])),
#     size=100,
#     window=10,
#     min_count=2,
#     workers=10,
#     iter=10,
#     sg = 1)

In [29]:
#all_model.save(f"model_all_0707.wordvectors")

## Training models per decade

In [33]:
#df per decade
df_40 = all_df.loc[(all_df['date'] >= '1940-01-01') & (all_df['date'] < '1950-01-01')]
df_50 = all_df.loc[(all_df['date'] >= '1950-01-01') & (all_df['date'] < '1960-01-01')]
df_60 = all_df.loc[(all_df['date'] >= '1960-01-01') & (all_df['date'] < '1970-01-01')]
df_70 = all_df.loc[(all_df['date'] >= '1970-01-01') & (all_df['date'] < '1980-01-01')]
df_80 = all_df.loc[(all_df['date'] >= '1980-01-01') & (all_df['date'] < '1990-01-01')]
df_90 = all_df.loc[(all_df['date'] >= '1990-01-01') & (all_df['date'] < '2000-01-01')]
print(len(df_40),len(df_50),len(df_60),len(df_70),len(df_80), len(df_90))

ERROR! Session/line number was not unique in database. History logging moved to new session 662
1762 6656 10809 15057 38996 40315


In [34]:
%%time
model_40 = gensim.models.Word2Vec(
    preprocess(list(df_40['text'])),
    size=100,
    window=10,
    min_count=5,
    workers=10,
    iter=10,)
model_40.save(f"model_40_2607.wordvectors")

INFO - 21:25:35: collecting all words and their counts
INFO - 21:25:35: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 21:25:36: collected 105289 word types from a corpus of 358056 raw words and 1762 sentences
INFO - 21:25:36: Loading a fresh vocabulary
INFO - 21:25:36: effective_min_count=5 retains 9295 unique words (8% of original 105289, drops 95994)
INFO - 21:25:36: effective_min_count=5 leaves 239039 word corpus (66% of original 358056, drops 119017)
INFO - 21:25:36: deleting the raw counts dictionary of 105289 items
INFO - 21:25:36: sample=0.001 downsamples 14 most-common words
INFO - 21:25:36: downsampling leaves estimated 236366 word corpus (98.9% of prior 239039)
INFO - 21:25:36: estimated required memory for 9295 words and 100 dimensions: 12083500 bytes
INFO - 21:25:36: resetting layer weights
INFO - 21:25:38: training model with 10 workers on 9295 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
INFO - 21:25:38: worker 

INFO - 21:25:41: worker thread finished; awaiting finish of 5 more threads
INFO - 21:25:41: worker thread finished; awaiting finish of 4 more threads
INFO - 21:25:41: worker thread finished; awaiting finish of 3 more threads
INFO - 21:25:41: worker thread finished; awaiting finish of 2 more threads
INFO - 21:25:41: worker thread finished; awaiting finish of 1 more threads
INFO - 21:25:41: worker thread finished; awaiting finish of 0 more threads
INFO - 21:25:41: EPOCH - 9 : training on 358056 raw words (236338 effective words) took 0.4s, 579045 effective words/s
INFO - 21:25:41: worker thread finished; awaiting finish of 9 more threads
INFO - 21:25:41: worker thread finished; awaiting finish of 8 more threads
INFO - 21:25:41: worker thread finished; awaiting finish of 7 more threads
INFO - 21:25:41: worker thread finished; awaiting finish of 6 more threads
INFO - 21:25:41: worker thread finished; awaiting finish of 5 more threads
INFO - 21:25:41: worker thread finished; awaiting finish

CPU times: user 2min 49s, sys: 13 s, total: 3min 2s
Wall time: 3min


In [35]:
%%time
model_50 = gensim.models.Word2Vec(
    preprocess(list(df_50['text'])),
    size=100,
    window=10,
    min_count=5,
    workers=10,
    iter=10,)
model_50.save(f"model_50_2607.wordvectors")

INFO - 21:34:49: collecting all words and their counts
INFO - 21:34:50: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 21:34:51: collected 333852 word types from a corpus of 1447068 raw words and 6655 sentences
INFO - 21:34:51: Loading a fresh vocabulary
INFO - 21:34:51: effective_min_count=5 retains 28294 unique words (8% of original 333852, drops 305558)
INFO - 21:34:51: effective_min_count=5 leaves 1074309 word corpus (74% of original 1447068, drops 372759)
INFO - 21:34:51: deleting the raw counts dictionary of 333852 items
INFO - 21:34:51: sample=0.001 downsamples 10 most-common words
INFO - 21:34:51: downsampling leaves estimated 1061767 word corpus (98.8% of prior 1074309)
INFO - 21:34:52: estimated required memory for 28294 words and 100 dimensions: 36782200 bytes
INFO - 21:34:52: resetting layer weights
INFO - 21:35:02: training model with 10 workers on 28294 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
INFO - 21:35:03

INFO - 21:35:20: EPOCH - 7 : training on 1447068 raw words (1061805 effective words) took 2.7s, 390451 effective words/s
INFO - 21:35:21: EPOCH 8 - PROGRESS: at 35.58% examples, 359819 words/s, in_qsize 18, out_qsize 1
INFO - 21:35:22: EPOCH 8 - PROGRESS: at 71.80% examples, 358035 words/s, in_qsize 20, out_qsize 0
INFO - 21:35:23: worker thread finished; awaiting finish of 9 more threads
INFO - 21:35:23: worker thread finished; awaiting finish of 8 more threads
INFO - 21:35:23: worker thread finished; awaiting finish of 7 more threads
INFO - 21:35:23: worker thread finished; awaiting finish of 6 more threads
INFO - 21:35:23: worker thread finished; awaiting finish of 5 more threads
INFO - 21:35:23: worker thread finished; awaiting finish of 4 more threads
INFO - 21:35:23: worker thread finished; awaiting finish of 3 more threads
INFO - 21:35:23: worker thread finished; awaiting finish of 2 more threads
INFO - 21:35:23: worker thread finished; awaiting finish of 1 more threads
INFO - 2

CPU times: user 8min 51s, sys: 54 s, total: 9min 45s
Wall time: 9min 46s


In [36]:
%%time
model_60 = gensim.models.Word2Vec(
    preprocess(list(df_60['text'])),
    size=100,
    window=10,
    min_count=5,
    workers=10,
    iter=10,)
model_60.save(f"model_60_2607.wordvectors")

INFO - 21:47:54: collecting all words and their counts
INFO - 21:47:54: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 21:47:55: PROGRESS: at sentence #10000, processed 2207887 words, keeping 443941 word types
INFO - 21:47:55: collected 467929 word types from a corpus of 2374030 raw words and 10806 sentences
INFO - 21:47:55: Loading a fresh vocabulary
INFO - 21:47:55: effective_min_count=5 retains 40969 unique words (8% of original 467929, drops 426960)
INFO - 21:47:55: effective_min_count=5 leaves 1851762 word corpus (78% of original 2374030, drops 522268)
INFO - 21:47:55: deleting the raw counts dictionary of 467929 items
INFO - 21:47:55: sample=0.001 downsamples 8 most-common words
INFO - 21:47:55: downsampling leaves estimated 1841183 word corpus (99.4% of prior 1851762)
INFO - 21:47:55: estimated required memory for 40969 words and 100 dimensions: 53259700 bytes
INFO - 21:47:55: resetting layer weights
INFO - 21:48:04: training model with 10 workers on 40

INFO - 21:48:25: worker thread finished; awaiting finish of 4 more threads
INFO - 21:48:25: worker thread finished; awaiting finish of 3 more threads
INFO - 21:48:25: worker thread finished; awaiting finish of 2 more threads
INFO - 21:48:25: worker thread finished; awaiting finish of 1 more threads
INFO - 21:48:25: worker thread finished; awaiting finish of 0 more threads
INFO - 21:48:25: EPOCH - 7 : training on 2374030 raw words (1841297 effective words) took 2.7s, 687520 effective words/s
INFO - 21:48:26: EPOCH 8 - PROGRESS: at 40.48% examples, 692324 words/s, in_qsize 19, out_qsize 0
INFO - 21:48:27: EPOCH 8 - PROGRESS: at 82.01% examples, 742049 words/s, in_qsize 19, out_qsize 0
INFO - 21:48:27: worker thread finished; awaiting finish of 9 more threads
INFO - 21:48:28: worker thread finished; awaiting finish of 8 more threads
INFO - 21:48:28: worker thread finished; awaiting finish of 7 more threads
INFO - 21:48:28: worker thread finished; awaiting finish of 6 more threads
INFO - 2

CPU times: user 12min 16s, sys: 1min 13s, total: 13min 30s
Wall time: 13min 4s


In [37]:
%%time
model_70 = gensim.models.Word2Vec(
    preprocess(list(df_70['text'])),
    size=100,
    window=10,
    min_count=5,
    workers=10,
    iter=10,)
model_70.save(f"model_70_2607.wordvectors")

INFO - 22:09:14: collecting all words and their counts
INFO - 22:09:14: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 22:09:15: PROGRESS: at sentence #10000, processed 3029830 words, keeping 524450 word types
INFO - 22:09:16: collected 754821 word types from a corpus of 4794235 raw words and 15057 sentences
INFO - 22:09:16: Loading a fresh vocabulary
INFO - 22:09:16: effective_min_count=5 retains 68516 unique words (9% of original 754821, drops 686305)
INFO - 22:09:16: effective_min_count=5 leaves 3956351 word corpus (82% of original 4794235, drops 837884)
INFO - 22:09:17: deleting the raw counts dictionary of 754821 items
INFO - 22:09:17: sample=0.001 downsamples 10 most-common words
INFO - 22:09:17: downsampling leaves estimated 3930516 word corpus (99.3% of prior 3956351)
INFO - 22:09:17: estimated required memory for 68516 words and 100 dimensions: 89070800 bytes
INFO - 22:09:17: resetting layer weights
INFO - 22:09:36: training model with 10 workers on 6

INFO - 22:10:08: EPOCH 6 - PROGRESS: at 91.32% examples, 701898 words/s, in_qsize 17, out_qsize 4
INFO - 22:10:09: worker thread finished; awaiting finish of 9 more threads
INFO - 22:10:09: worker thread finished; awaiting finish of 8 more threads
INFO - 22:10:09: worker thread finished; awaiting finish of 7 more threads
INFO - 22:10:09: worker thread finished; awaiting finish of 6 more threads
INFO - 22:10:09: worker thread finished; awaiting finish of 5 more threads
INFO - 22:10:09: worker thread finished; awaiting finish of 4 more threads
INFO - 22:10:09: worker thread finished; awaiting finish of 3 more threads
INFO - 22:10:09: worker thread finished; awaiting finish of 2 more threads
INFO - 22:10:09: worker thread finished; awaiting finish of 1 more threads
INFO - 22:10:09: worker thread finished; awaiting finish of 0 more threads
INFO - 22:10:09: EPOCH - 6 : training on 4794235 raw words (3930567 effective words) took 5.9s, 663360 effective words/s
INFO - 22:10:10: EPOCH 7 - PROG

CPU times: user 21min 32s, sys: 2min 14s, total: 23min 47s
Wall time: 22min


In [38]:
%%time
model_80 = gensim.models.Word2Vec(
    preprocess(list(df_80['text'])),
    size=100,
    window=10,
    min_count=5,
    workers=10,
    iter=10,)
model_80.save(f"model_80_2607.wordvectors")

INFO - 22:49:57: collecting all words and their counts
INFO - 22:49:57: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 22:49:59: PROGRESS: at sentence #10000, processed 3333594 words, keeping 495957 word types
INFO - 22:50:00: PROGRESS: at sentence #20000, processed 6560901 words, keeping 762436 word types
INFO - 22:50:01: PROGRESS: at sentence #30000, processed 9529278 words, keeping 992367 word types
INFO - 22:50:01: collected 1146270 word types from a corpus of 11984205 raw words and 38991 sentences
INFO - 22:50:01: Loading a fresh vocabulary
INFO - 22:50:03: effective_min_count=5 retains 123315 unique words (10% of original 1146270, drops 1022955)
INFO - 22:50:03: effective_min_count=5 leaves 10663585 word corpus (88% of original 11984205, drops 1320620)
INFO - 22:50:04: deleting the raw counts dictionary of 1146270 items
INFO - 22:50:04: sample=0.001 downsamples 13 most-common words
INFO - 22:50:04: downsampling leaves estimated 10515431 word corpus (98.6

INFO - 22:51:19: EPOCH - 3 : training on 11984205 raw words (10515361 effective words) took 15.9s, 660970 effective words/s
INFO - 22:51:20: EPOCH 4 - PROGRESS: at 5.68% examples, 617141 words/s, in_qsize 20, out_qsize 0
INFO - 22:51:21: EPOCH 4 - PROGRESS: at 12.63% examples, 663526 words/s, in_qsize 18, out_qsize 1
INFO - 22:51:22: EPOCH 4 - PROGRESS: at 18.32% examples, 662718 words/s, in_qsize 20, out_qsize 0
INFO - 22:51:23: EPOCH 4 - PROGRESS: at 24.55% examples, 674187 words/s, in_qsize 19, out_qsize 0
INFO - 22:51:24: EPOCH 4 - PROGRESS: at 31.23% examples, 680036 words/s, in_qsize 16, out_qsize 3
INFO - 22:51:25: EPOCH 4 - PROGRESS: at 37.75% examples, 682294 words/s, in_qsize 19, out_qsize 0
INFO - 22:51:26: EPOCH 4 - PROGRESS: at 43.84% examples, 684375 words/s, in_qsize 20, out_qsize 1
INFO - 22:51:27: EPOCH 4 - PROGRESS: at 50.36% examples, 686576 words/s, in_qsize 19, out_qsize 0
INFO - 22:51:28: EPOCH 4 - PROGRESS: at 56.76% examples, 683996 words/s, in_qsize 20, out_qsi

INFO - 22:52:19: EPOCH 7 - PROGRESS: at 98.56% examples, 729970 words/s, in_qsize 16, out_qsize 2
INFO - 22:52:19: worker thread finished; awaiting finish of 9 more threads
INFO - 22:52:19: worker thread finished; awaiting finish of 8 more threads
INFO - 22:52:19: worker thread finished; awaiting finish of 7 more threads
INFO - 22:52:19: worker thread finished; awaiting finish of 6 more threads
INFO - 22:52:19: worker thread finished; awaiting finish of 5 more threads
INFO - 22:52:19: worker thread finished; awaiting finish of 4 more threads
INFO - 22:52:19: worker thread finished; awaiting finish of 3 more threads
INFO - 22:52:19: worker thread finished; awaiting finish of 2 more threads
INFO - 22:52:19: worker thread finished; awaiting finish of 1 more threads
INFO - 22:52:19: worker thread finished; awaiting finish of 0 more threads
INFO - 22:52:19: EPOCH - 7 : training on 11984205 raw words (10515096 effective words) took 14.4s, 729119 effective words/s
INFO - 22:52:20: EPOCH 8 - P

INFO - 22:53:03: saved model_80_2607.wordvectors


CPU times: user 42min 27s, sys: 5min 26s, total: 47min 53s
Wall time: 42min 29s


In [39]:
%%time
model_90 = gensim.models.Word2Vec(
    preprocess(list(df_90['text'])),
    size=100,
    window=10,
    min_count=5,
    workers=10,
    iter=10,)
model_90.save(f"model_90_2607.wordvectors")

INFO - 23:25:52: collecting all words and their counts
INFO - 23:25:52: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 23:25:52: PROGRESS: at sentence #10000, processed 2615573 words, keeping 273579 word types
INFO - 23:25:53: PROGRESS: at sentence #20000, processed 5615570 words, keeping 490225 word types
INFO - 23:25:54: PROGRESS: at sentence #30000, processed 8691380 words, keeping 697001 word types
INFO - 23:25:55: PROGRESS: at sentence #40000, processed 11855528 words, keeping 887949 word types
INFO - 23:25:56: collected 894511 word types from a corpus of 11955553 raw words and 40313 sentences
INFO - 23:25:56: Loading a fresh vocabulary
INFO - 23:25:56: effective_min_count=5 retains 113668 unique words (12% of original 894511, drops 780843)
INFO - 23:25:56: effective_min_count=5 leaves 10923487 word corpus (91% of original 11955553, drops 1032066)
INFO - 23:25:56: deleting the raw counts dictionary of 894511 items
INFO - 23:25:56: sample=0.001 downsamples

INFO - 23:27:06: EPOCH 4 - PROGRESS: at 16.43% examples, 749896 words/s, in_qsize 19, out_qsize 0
INFO - 23:27:07: EPOCH 4 - PROGRESS: at 24.13% examples, 750466 words/s, in_qsize 19, out_qsize 0
INFO - 23:27:08: EPOCH 4 - PROGRESS: at 31.79% examples, 745366 words/s, in_qsize 17, out_qsize 2
INFO - 23:27:09: EPOCH 4 - PROGRESS: at 38.77% examples, 748430 words/s, in_qsize 19, out_qsize 0
INFO - 23:27:10: EPOCH 4 - PROGRESS: at 45.61% examples, 748971 words/s, in_qsize 17, out_qsize 2
INFO - 23:27:11: EPOCH 4 - PROGRESS: at 52.23% examples, 747344 words/s, in_qsize 17, out_qsize 2
INFO - 23:27:12: EPOCH 4 - PROGRESS: at 58.74% examples, 745690 words/s, in_qsize 16, out_qsize 3
INFO - 23:27:13: EPOCH 4 - PROGRESS: at 65.84% examples, 744524 words/s, in_qsize 16, out_qsize 3
INFO - 23:27:14: EPOCH 4 - PROGRESS: at 72.41% examples, 743688 words/s, in_qsize 16, out_qsize 3
INFO - 23:27:15: EPOCH 4 - PROGRESS: at 79.01% examples, 742126 words/s, in_qsize 20, out_qsize 0
INFO - 23:27:16: EPO

INFO - 23:28:02: worker thread finished; awaiting finish of 6 more threads
INFO - 23:28:02: worker thread finished; awaiting finish of 5 more threads
INFO - 23:28:02: worker thread finished; awaiting finish of 4 more threads
INFO - 23:28:02: worker thread finished; awaiting finish of 3 more threads
INFO - 23:28:02: worker thread finished; awaiting finish of 2 more threads
INFO - 23:28:02: worker thread finished; awaiting finish of 1 more threads
INFO - 23:28:02: worker thread finished; awaiting finish of 0 more threads
INFO - 23:28:02: EPOCH - 7 : training on 11955553 raw words (10741359 effective words) took 14.4s, 748084 effective words/s
INFO - 23:28:03: EPOCH 8 - PROGRESS: at 7.90% examples, 718092 words/s, in_qsize 19, out_qsize 0
INFO - 23:28:04: EPOCH 8 - PROGRESS: at 16.33% examples, 740844 words/s, in_qsize 18, out_qsize 1
INFO - 23:28:05: EPOCH 8 - PROGRESS: at 24.43% examples, 748296 words/s, in_qsize 20, out_qsize 1
INFO - 23:28:06: EPOCH 8 - PROGRESS: at 32.09% examples, 7

CPU times: user 35min 54s, sys: 5min 15s, total: 41min 9s
Wall time: 35min 46s


### Training models per decade no ads

In [40]:
#df per decade
na_df_40 = no_ads.loc[(no_ads['date'] >= '1940-01-01') & (no_ads['date'] < '1950-01-01')]
na_df_50 = no_ads.loc[(no_ads['date'] >= '1950-01-01') & (no_ads['date'] < '1960-01-01')]
na_df_60 = no_ads.loc[(no_ads['date'] >= '1960-01-01') & (no_ads['date'] < '1970-01-01')]
na_df_70 = no_ads.loc[(no_ads['date'] >= '1970-01-01') & (no_ads['date'] < '1980-01-01')]
na_df_80 = no_ads.loc[(no_ads['date'] >= '1980-01-01') & (no_ads['date'] < '1990-01-01')]
na_df_90 = no_ads.loc[(no_ads['date'] >= '1990-01-01') & (no_ads['date'] < '2000-01-01')]
print(len(df_40),len(df_50),len(df_60),len(df_70),len(df_80), len(df_90))

1762 6656 10809 15057 38996 40315


In [41]:
%%time
na_model_40 = gensim.models.Word2Vec(
    preprocess(list(na_df_40['text'])),
    size=100,
    window=10,
    min_count=5,
    workers=10,
    iter=10,)
na_model_40.save(f"na_model_40_2607.wordvectors")

INFO - 23:29:31: collecting all words and their counts
INFO - 23:29:31: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 23:29:31: collected 72230 word types from a corpus of 275269 raw words and 1371 sentences
INFO - 23:29:31: Loading a fresh vocabulary
INFO - 23:29:31: effective_min_count=5 retains 7774 unique words (10% of original 72230, drops 64456)
INFO - 23:29:31: effective_min_count=5 leaves 192403 word corpus (69% of original 275269, drops 82866)
INFO - 23:29:31: deleting the raw counts dictionary of 72230 items
INFO - 23:29:31: sample=0.001 downsamples 17 most-common words
INFO - 23:29:31: downsampling leaves estimated 189959 word corpus (98.7% of prior 192403)
INFO - 23:29:31: estimated required memory for 7774 words and 100 dimensions: 10106200 bytes
INFO - 23:29:31: resetting layer weights
INFO - 23:29:33: training model with 10 workers on 7774 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
INFO - 23:29:33: worker thr

INFO - 23:29:35: worker thread finished; awaiting finish of 5 more threads
INFO - 23:29:35: worker thread finished; awaiting finish of 4 more threads
INFO - 23:29:35: worker thread finished; awaiting finish of 3 more threads
INFO - 23:29:35: worker thread finished; awaiting finish of 2 more threads
INFO - 23:29:35: worker thread finished; awaiting finish of 1 more threads
INFO - 23:29:35: worker thread finished; awaiting finish of 0 more threads
INFO - 23:29:35: EPOCH - 9 : training on 275269 raw words (189955 effective words) took 0.3s, 716911 effective words/s
INFO - 23:29:36: worker thread finished; awaiting finish of 9 more threads
INFO - 23:29:36: worker thread finished; awaiting finish of 8 more threads
INFO - 23:29:36: worker thread finished; awaiting finish of 7 more threads
INFO - 23:29:36: worker thread finished; awaiting finish of 6 more threads
INFO - 23:29:36: worker thread finished; awaiting finish of 5 more threads
INFO - 23:29:36: worker thread finished; awaiting finish

CPU times: user 35.7 s, sys: 9.04 s, total: 44.8 s
Wall time: 42.1 s


In [42]:
%%time
na_model_50 = gensim.models.Word2Vec(
    preprocess(list(na_df_50['text'])),
    size=100,
    window=10,
    min_count=5,
    workers=10,
    iter=10,)
na_model_50.save(f"na_model_50_2607.wordvectors")

INFO - 23:32:19: collecting all words and their counts
INFO - 23:32:19: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 23:32:19: collected 284740 word types from a corpus of 1282505 raw words and 5632 sentences
INFO - 23:32:19: Loading a fresh vocabulary
INFO - 23:32:19: effective_min_count=5 retains 25650 unique words (9% of original 284740, drops 259090)
INFO - 23:32:19: effective_min_count=5 leaves 963338 word corpus (75% of original 1282505, drops 319167)
INFO - 23:32:19: deleting the raw counts dictionary of 284740 items
INFO - 23:32:19: sample=0.001 downsamples 11 most-common words
INFO - 23:32:19: downsampling leaves estimated 950453 word corpus (98.7% of prior 963338)
INFO - 23:32:20: estimated required memory for 25650 words and 100 dimensions: 33345000 bytes
INFO - 23:32:20: resetting layer weights
INFO - 23:32:25: training model with 10 workers on 25650 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
INFO - 23:32:26: E

INFO - 23:32:35: worker thread finished; awaiting finish of 4 more threads
INFO - 23:32:35: worker thread finished; awaiting finish of 3 more threads
INFO - 23:32:35: worker thread finished; awaiting finish of 2 more threads
INFO - 23:32:35: worker thread finished; awaiting finish of 1 more threads
INFO - 23:32:35: worker thread finished; awaiting finish of 0 more threads
INFO - 23:32:35: EPOCH - 8 : training on 1282505 raw words (950315 effective words) took 1.3s, 756423 effective words/s
INFO - 23:32:36: EPOCH 9 - PROGRESS: at 83.88% examples, 770847 words/s, in_qsize 17, out_qsize 2
INFO - 23:32:36: worker thread finished; awaiting finish of 9 more threads
INFO - 23:32:36: worker thread finished; awaiting finish of 8 more threads
INFO - 23:32:36: worker thread finished; awaiting finish of 7 more threads
INFO - 23:32:36: worker thread finished; awaiting finish of 6 more threads
INFO - 23:32:36: worker thread finished; awaiting finish of 5 more threads
INFO - 23:32:36: worker thread f

CPU times: user 2min 47s, sys: 39.9 s, total: 3min 27s
Wall time: 3min 2s


In [43]:
%%time
na_model_60 = gensim.models.Word2Vec(
    preprocess(list(na_df_60['text'])),
    size=100,
    window=10,
    min_count=5,
    workers=10,
    iter=10,)
na_model_60.save(f"na_model_60_2607.wordvectors")

INFO - 23:36:32: collecting all words and their counts
INFO - 23:36:32: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 23:36:32: collected 366320 word types from a corpus of 1996811 raw words and 9082 sentences
INFO - 23:36:32: Loading a fresh vocabulary
INFO - 23:36:33: effective_min_count=5 retains 35506 unique words (9% of original 366320, drops 330814)
INFO - 23:36:33: effective_min_count=5 leaves 1587019 word corpus (79% of original 1996811, drops 409792)
INFO - 23:36:33: deleting the raw counts dictionary of 366320 items
INFO - 23:36:33: sample=0.001 downsamples 10 most-common words
INFO - 23:36:33: downsampling leaves estimated 1576100 word corpus (99.3% of prior 1587019)
INFO - 23:36:33: estimated required memory for 35506 words and 100 dimensions: 46157800 bytes
INFO - 23:36:33: resetting layer weights
INFO - 23:36:40: training model with 10 workers on 35506 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
INFO - 23:36:41

INFO - 23:36:58: worker thread finished; awaiting finish of 0 more threads
INFO - 23:36:58: EPOCH - 7 : training on 1996811 raw words (1576064 effective words) took 2.8s, 559265 effective words/s
INFO - 23:36:59: EPOCH 8 - PROGRESS: at 38.70% examples, 551978 words/s, in_qsize 18, out_qsize 1
INFO - 23:37:00: EPOCH 8 - PROGRESS: at 76.29% examples, 576370 words/s, in_qsize 19, out_qsize 0
INFO - 23:37:01: worker thread finished; awaiting finish of 9 more threads
INFO - 23:37:01: worker thread finished; awaiting finish of 8 more threads
INFO - 23:37:01: worker thread finished; awaiting finish of 7 more threads
INFO - 23:37:01: worker thread finished; awaiting finish of 6 more threads
INFO - 23:37:01: worker thread finished; awaiting finish of 5 more threads
INFO - 23:37:01: worker thread finished; awaiting finish of 4 more threads
INFO - 23:37:01: worker thread finished; awaiting finish of 3 more threads
INFO - 23:37:01: worker thread finished; awaiting finish of 2 more threads
INFO - 2

CPU times: user 4min 15s, sys: 54.8 s, total: 5min 9s
Wall time: 4min 27s


In [44]:
%%time
na_model_70 = gensim.models.Word2Vec(
    preprocess(list(na_df_70['text'])),
    size=100,
    window=10,
    min_count=5,
    workers=10,
    iter=10,)
na_model_70.save(f"na_model_70_2607.wordvectors")

INFO - 23:44:05: collecting all words and their counts
INFO - 23:44:05: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 23:44:06: PROGRESS: at sentence #10000, processed 3198065 words, keeping 463303 word types
INFO - 23:44:06: collected 514945 word types from a corpus of 3763490 raw words and 11679 sentences
INFO - 23:44:06: Loading a fresh vocabulary
INFO - 23:44:06: effective_min_count=5 retains 55468 unique words (10% of original 514945, drops 459477)
INFO - 23:44:06: effective_min_count=5 leaves 3189861 word corpus (84% of original 3763490, drops 573629)
INFO - 23:44:07: deleting the raw counts dictionary of 514945 items
INFO - 23:44:07: sample=0.001 downsamples 14 most-common words
INFO - 23:44:07: downsampling leaves estimated 3160914 word corpus (99.1% of prior 3189861)
INFO - 23:44:07: estimated required memory for 55468 words and 100 dimensions: 72108400 bytes
INFO - 23:44:07: resetting layer weights
INFO - 23:44:18: training model with 10 workers on 

INFO - 23:44:44: worker thread finished; awaiting finish of 3 more threads
INFO - 23:44:44: worker thread finished; awaiting finish of 2 more threads
INFO - 23:44:44: worker thread finished; awaiting finish of 1 more threads
INFO - 23:44:44: worker thread finished; awaiting finish of 0 more threads
INFO - 23:44:44: EPOCH - 6 : training on 3763490 raw words (3161048 effective words) took 4.3s, 738771 effective words/s
INFO - 23:44:45: EPOCH 7 - PROGRESS: at 29.24% examples, 709230 words/s, in_qsize 16, out_qsize 3
INFO - 23:44:46: EPOCH 7 - PROGRESS: at 56.48% examples, 743542 words/s, in_qsize 16, out_qsize 4
INFO - 23:44:47: EPOCH 7 - PROGRESS: at 75.13% examples, 746065 words/s, in_qsize 20, out_qsize 2
INFO - 23:44:48: EPOCH 7 - PROGRESS: at 96.61% examples, 741999 words/s, in_qsize 10, out_qsize 4
INFO - 23:44:48: worker thread finished; awaiting finish of 9 more threads
INFO - 23:44:48: worker thread finished; awaiting finish of 8 more threads
INFO - 23:44:48: worker thread finish

CPU times: user 7min 40s, sys: 1min 41s, total: 9min 21s
Wall time: 7min 55s


In [45]:
%%time
na_model_80 = gensim.models.Word2Vec(
    preprocess(list(na_df_80['text'])),
    size=100,
    window=10,
    min_count=5,
    workers=10,
    iter=10,)
na_model_80.save(f"na_model_80_2607.wordvectors")

INFO - 23:56:40: collecting all words and their counts
INFO - 23:56:40: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 23:56:41: PROGRESS: at sentence #10000, processed 3489293 words, keeping 402576 word types
INFO - 23:56:42: collected 611556 word types from a corpus of 6657712 raw words and 19202 sentences
INFO - 23:56:42: Loading a fresh vocabulary
INFO - 23:56:42: effective_min_count=5 retains 79635 unique words (13% of original 611556, drops 531921)
INFO - 23:56:42: effective_min_count=5 leaves 5970097 word corpus (89% of original 6657712, drops 687615)
INFO - 23:56:42: deleting the raw counts dictionary of 611556 items
INFO - 23:56:42: sample=0.001 downsamples 10 most-common words
INFO - 23:56:42: downsampling leaves estimated 5926061 word corpus (99.3% of prior 5970097)
INFO - 23:56:43: estimated required memory for 79635 words and 100 dimensions: 103525500 bytes
INFO - 23:56:43: resetting layer weights
INFO - 23:56:59: training model with 10 workers on

INFO - 23:57:39: worker thread finished; awaiting finish of 6 more threads
INFO - 23:57:39: worker thread finished; awaiting finish of 5 more threads
INFO - 23:57:39: worker thread finished; awaiting finish of 4 more threads
INFO - 23:57:39: worker thread finished; awaiting finish of 3 more threads
INFO - 23:57:39: worker thread finished; awaiting finish of 2 more threads
INFO - 23:57:39: worker thread finished; awaiting finish of 1 more threads
INFO - 23:57:39: worker thread finished; awaiting finish of 0 more threads
INFO - 23:57:39: EPOCH - 5 : training on 6657712 raw words (5926218 effective words) took 8.0s, 744876 effective words/s
INFO - 23:57:40: EPOCH 6 - PROGRESS: at 13.07% examples, 701552 words/s, in_qsize 18, out_qsize 1
INFO - 23:57:41: EPOCH 6 - PROGRESS: at 26.27% examples, 731847 words/s, in_qsize 17, out_qsize 2
INFO - 23:57:42: EPOCH 6 - PROGRESS: at 39.54% examples, 745265 words/s, in_qsize 19, out_qsize 0
INFO - 23:57:43: EPOCH 6 - PROGRESS: at 51.86% examples, 750

INFO - 23:58:19: worker thread finished; awaiting finish of 0 more threads
INFO - 23:58:19: EPOCH - 10 : training on 6657712 raw words (5925997 effective words) took 8.0s, 738280 effective words/s
INFO - 23:58:19: training on a 66577120 raw words (59260692 effective words) took 79.7s, 743746 effective words/s
INFO - 23:58:19: saving Word2Vec object under na_model_80_2607.wordvectors, separately None
INFO - 23:58:19: not storing attribute vectors_norm
INFO - 23:58:19: not storing attribute cum_table
INFO - 23:58:20: saved na_model_80_2607.wordvectors


CPU times: user 13min 18s, sys: 2min 51s, total: 16min 10s
Wall time: 13min 18s


In [46]:
#preprocess(list(na_df_90['text']))
na_df_90['text']

46973    ['\nVan Basten neemt wraak\nVan onze Telesport...
46974    ['\nMilan dank zij treffer Van Basten dichter ...
46975    ['\nBUITENLANDS VOETBAL\nENGELAND: Aston Villa...
46976    ["\nVan 't Hek maakt Gaaikema goed door Ruud V...
46977    ['\ntelesport Buitenlands Voetbal in Cijfers\n...
                               ...                        
61855    ["\nEngeland 2\n08.30 The adventures of Skippy...
61856    ["\nGOEDE SPELEN, SLECHTE SPELEN door Jan Joos...
61857    ["\nTELEVISIE\n■ NEDERLAND 1 07.00 NOS: Tekst ...
61858    ['\nGoloebjeva - Van Lith (WK-vrouwen 1995)\n$...
61859    ['\nVOETBAL\nTRIPOLI: vriendschappelijk toerno...
Name: text, Length: 14887, dtype: object

In [47]:
%%time
na_model_90 = gensim.models.Word2Vec(
    preprocess(list(na_df_90['text'])),
    size=100,
    window=10,
    min_count=5,
    workers=10,
    iter=10,)
na_model_90.save(f"n_model_90_2607.wordvectors")

INFO - 00:07:44: collecting all words and their counts
INFO - 00:07:44: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 00:07:45: PROGRESS: at sentence #10000, processed 3710002 words, keeping 351328 word types
INFO - 00:07:46: collected 472376 word types from a corpus of 5569560 raw words and 14887 sentences
INFO - 00:07:46: Loading a fresh vocabulary
INFO - 00:07:46: effective_min_count=5 retains 70929 unique words (15% of original 472376, drops 401447)
INFO - 00:07:46: effective_min_count=5 leaves 5036732 word corpus (90% of original 5569560, drops 532828)
INFO - 00:07:46: deleting the raw counts dictionary of 472376 items
INFO - 00:07:46: sample=0.001 downsamples 9 most-common words
INFO - 00:07:46: downsampling leaves estimated 5002558 word corpus (99.3% of prior 5036732)
INFO - 00:07:46: estimated required memory for 70929 words and 100 dimensions: 92207700 bytes
INFO - 00:07:46: resetting layer weights
INFO - 00:08:01: training model with 10 workers on 7

INFO - 00:08:35: EPOCH - 5 : training on 5569560 raw words (5002598 effective words) took 7.0s, 718648 effective words/s
INFO - 00:08:36: EPOCH 6 - PROGRESS: at 15.91% examples, 702975 words/s, in_qsize 17, out_qsize 2
INFO - 00:08:37: EPOCH 6 - PROGRESS: at 31.79% examples, 735845 words/s, in_qsize 18, out_qsize 1
INFO - 00:08:38: EPOCH 6 - PROGRESS: at 45.93% examples, 733019 words/s, in_qsize 19, out_qsize 0
INFO - 00:08:39: EPOCH 6 - PROGRESS: at 61.27% examples, 734239 words/s, in_qsize 18, out_qsize 1
INFO - 00:08:40: EPOCH 6 - PROGRESS: at 75.53% examples, 734260 words/s, in_qsize 19, out_qsize 0
INFO - 00:08:41: EPOCH 6 - PROGRESS: at 89.90% examples, 736445 words/s, in_qsize 19, out_qsize 0
INFO - 00:08:42: worker thread finished; awaiting finish of 9 more threads
INFO - 00:08:42: worker thread finished; awaiting finish of 8 more threads
INFO - 00:08:42: worker thread finished; awaiting finish of 7 more threads
INFO - 00:08:42: worker thread finished; awaiting finish of 6 more

CPU times: user 10min 52s, sys: 2min 19s, total: 13min 12s
Wall time: 10min 50s


## Training models per newspaper

In [48]:
#df per krant
df_trouw = all_df.loc[all_df['ppn'] == 412789353]
df_vk = all_df.loc[all_df['ppn'] == 412869594]
df_tel = all_df.loc[all_df['ppn'] == 832675288]
df_nrc = all_df.loc[all_df['ppn'] == 400367629]


In [49]:
model_trouw = gensim.models.Word2Vec(
    preprocess(list(df_trouw['text'])),
    size=100,
    window=10,
    min_count=5,
    workers=10,
    iter=10,)
model_trouw.save(f"model_trouw_2607.wordvectors")

INFO - 00:20:15: collecting all words and their counts
INFO - 00:20:15: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 00:20:16: PROGRESS: at sentence #10000, processed 2142365 words, keeping 394069 word types
INFO - 00:20:17: PROGRESS: at sentence #20000, processed 4333970 words, keeping 593295 word types
INFO - 00:20:18: collected 730521 word types from a corpus of 5924903 raw words and 25564 sentences
INFO - 00:20:18: Loading a fresh vocabulary
INFO - 00:20:18: effective_min_count=5 retains 72871 unique words (9% of original 730521, drops 657650)
INFO - 00:20:18: effective_min_count=5 leaves 5099663 word corpus (86% of original 5924903, drops 825240)
INFO - 00:20:18: deleting the raw counts dictionary of 730521 items
INFO - 00:20:18: sample=0.001 downsamples 13 most-common words
INFO - 00:20:18: downsampling leaves estimated 4973433 word corpus (97.5% of prior 5099663)
INFO - 00:20:18: estimated required memory for 72871 words and 100 dimensions: 94732300 b

INFO - 00:21:08: worker thread finished; awaiting finish of 1 more threads
INFO - 00:21:08: worker thread finished; awaiting finish of 0 more threads
INFO - 00:21:08: EPOCH - 5 : training on 5924903 raw words (4973783 effective words) took 6.9s, 722224 effective words/s
INFO - 00:21:09: EPOCH 6 - PROGRESS: at 16.89% examples, 689621 words/s, in_qsize 18, out_qsize 1
INFO - 00:21:10: EPOCH 6 - PROGRESS: at 33.13% examples, 714215 words/s, in_qsize 19, out_qsize 0
INFO - 00:21:11: EPOCH 6 - PROGRESS: at 50.00% examples, 728475 words/s, in_qsize 19, out_qsize 0
INFO - 00:21:12: EPOCH 6 - PROGRESS: at 65.28% examples, 722345 words/s, in_qsize 20, out_qsize 1
INFO - 00:21:13: EPOCH 6 - PROGRESS: at 78.83% examples, 721407 words/s, in_qsize 20, out_qsize 0
INFO - 00:21:14: EPOCH 6 - PROGRESS: at 90.98% examples, 727982 words/s, in_qsize 18, out_qsize 1
INFO - 00:21:14: worker thread finished; awaiting finish of 9 more threads
INFO - 00:21:14: worker thread finished; awaiting finish of 8 more

In [50]:
model_vk = gensim.models.Word2Vec(
    preprocess(list(df_vk['text'])),
    size=100,
    window=10,
    min_count=5,
    workers=10,
    iter=10,)
model_vk.save(f"model_vk_2607.wordvectors")

INFO - 00:40:26: collecting all words and their counts
INFO - 00:40:26: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 00:40:27: PROGRESS: at sentence #10000, processed 2775042 words, keeping 521846 word types
INFO - 00:40:29: PROGRESS: at sentence #20000, processed 5958679 words, keeping 883103 word types
INFO - 00:40:30: PROGRESS: at sentence #30000, processed 9769748 words, keeping 1214315 word types
INFO - 00:40:30: collected 1249409 word types from a corpus of 10254181 raw words and 31286 sentences
INFO - 00:40:30: Loading a fresh vocabulary
INFO - 00:40:31: effective_min_count=5 retains 112943 unique words (9% of original 1249409, drops 1136466)
INFO - 00:40:31: effective_min_count=5 leaves 8854886 word corpus (86% of original 10254181, drops 1399295)
INFO - 00:40:31: deleting the raw counts dictionary of 1249409 items
INFO - 00:40:31: sample=0.001 downsamples 11 most-common words
INFO - 00:40:31: downsampling leaves estimated 8706722 word corpus (98.3% 

INFO - 00:41:42: EPOCH 4 - PROGRESS: at 92.85% examples, 714933 words/s, in_qsize 20, out_qsize 0
INFO - 00:41:43: worker thread finished; awaiting finish of 9 more threads
INFO - 00:41:43: worker thread finished; awaiting finish of 8 more threads
INFO - 00:41:43: EPOCH 4 - PROGRESS: at 99.48% examples, 713910 words/s, in_qsize 6, out_qsize 3
INFO - 00:41:43: worker thread finished; awaiting finish of 7 more threads
INFO - 00:41:43: worker thread finished; awaiting finish of 6 more threads
INFO - 00:41:43: worker thread finished; awaiting finish of 5 more threads
INFO - 00:41:43: worker thread finished; awaiting finish of 4 more threads
INFO - 00:41:43: worker thread finished; awaiting finish of 3 more threads
INFO - 00:41:43: worker thread finished; awaiting finish of 2 more threads
INFO - 00:41:43: worker thread finished; awaiting finish of 1 more threads
INFO - 00:41:43: worker thread finished; awaiting finish of 0 more threads
INFO - 00:41:43: EPOCH - 4 : training on 10254181 raw w

INFO - 00:42:31: worker thread finished; awaiting finish of 6 more threads
INFO - 00:42:31: worker thread finished; awaiting finish of 5 more threads
INFO - 00:42:31: worker thread finished; awaiting finish of 4 more threads
INFO - 00:42:31: worker thread finished; awaiting finish of 3 more threads
INFO - 00:42:32: worker thread finished; awaiting finish of 2 more threads
INFO - 00:42:32: worker thread finished; awaiting finish of 1 more threads
INFO - 00:42:32: worker thread finished; awaiting finish of 0 more threads
INFO - 00:42:32: EPOCH - 8 : training on 10254181 raw words (8706248 effective words) took 12.0s, 726564 effective words/s
INFO - 00:42:33: EPOCH 9 - PROGRESS: at 10.90% examples, 677137 words/s, in_qsize 16, out_qsize 3
INFO - 00:42:34: EPOCH 9 - PROGRESS: at 22.02% examples, 717741 words/s, in_qsize 19, out_qsize 0
INFO - 00:42:35: EPOCH 9 - PROGRESS: at 30.96% examples, 719206 words/s, in_qsize 20, out_qsize 0
INFO - 00:42:36: EPOCH 9 - PROGRESS: at 40.46% examples, 7

In [51]:
model_tel = gensim.models.Word2Vec(
    preprocess(list(df_tel['text'])),
    size=100,
    window=10,
    min_count=5,
    workers=10,
    iter=10,)
model_tel.save(f"model_tel_2607.wordvectors")

INFO - 01:01:26: collecting all words and their counts
INFO - 01:01:26: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 01:01:27: PROGRESS: at sentence #10000, processed 1929070 words, keeping 481383 word types
INFO - 01:01:28: PROGRESS: at sentence #20000, processed 4823161 words, keeping 905219 word types
INFO - 01:01:29: PROGRESS: at sentence #30000, processed 6714008 words, keeping 1009084 word types
INFO - 01:01:30: PROGRESS: at sentence #40000, processed 8965002 words, keeping 1109360 word types
INFO - 01:01:30: collected 1138188 word types from a corpus of 9825336 raw words and 43525 sentences
INFO - 01:01:30: Loading a fresh vocabulary
INFO - 01:01:30: effective_min_count=5 retains 100169 unique words (8% of original 1138188, drops 1038019)
INFO - 01:01:30: effective_min_count=5 leaves 8563145 word corpus (87% of original 9825336, drops 1262191)
INFO - 01:01:31: deleting the raw counts dictionary of 1138188 items
INFO - 01:01:31: sample=0.001 downsample

INFO - 01:02:38: EPOCH 4 - PROGRESS: at 97.29% examples, 720410 words/s, in_qsize 18, out_qsize 1
INFO - 01:02:38: worker thread finished; awaiting finish of 9 more threads
INFO - 01:02:38: worker thread finished; awaiting finish of 8 more threads
INFO - 01:02:38: worker thread finished; awaiting finish of 7 more threads
INFO - 01:02:38: worker thread finished; awaiting finish of 6 more threads
INFO - 01:02:38: worker thread finished; awaiting finish of 5 more threads
INFO - 01:02:38: worker thread finished; awaiting finish of 4 more threads
INFO - 01:02:38: worker thread finished; awaiting finish of 3 more threads
INFO - 01:02:38: worker thread finished; awaiting finish of 2 more threads
INFO - 01:02:38: worker thread finished; awaiting finish of 1 more threads
INFO - 01:02:38: worker thread finished; awaiting finish of 0 more threads
INFO - 01:02:38: EPOCH - 4 : training on 9825336 raw words (8330773 effective words) took 11.6s, 717041 effective words/s
INFO - 01:02:39: EPOCH 5 - PRO

INFO - 01:03:25: worker thread finished; awaiting finish of 5 more threads
INFO - 01:03:25: worker thread finished; awaiting finish of 4 more threads
INFO - 01:03:25: worker thread finished; awaiting finish of 3 more threads
INFO - 01:03:25: worker thread finished; awaiting finish of 2 more threads
INFO - 01:03:25: worker thread finished; awaiting finish of 1 more threads
INFO - 01:03:25: worker thread finished; awaiting finish of 0 more threads
INFO - 01:03:25: EPOCH - 8 : training on 9825336 raw words (8331279 effective words) took 11.8s, 706354 effective words/s
INFO - 01:03:26: EPOCH 9 - PROGRESS: at 11.35% examples, 678955 words/s, in_qsize 18, out_qsize 1
INFO - 01:03:27: EPOCH 9 - PROGRESS: at 21.97% examples, 681605 words/s, in_qsize 20, out_qsize 1
INFO - 01:03:28: EPOCH 9 - PROGRESS: at 30.12% examples, 685579 words/s, in_qsize 19, out_qsize 1
INFO - 01:03:29: EPOCH 9 - PROGRESS: at 36.78% examples, 695680 words/s, in_qsize 19, out_qsize 0
INFO - 01:03:30: EPOCH 9 - PROGRESS:

In [52]:
model_nrc = gensim.models.Word2Vec(
    preprocess(list(df_nrc['text'])),
    size=100,
    window=10,
    min_count=5,
    workers=10,
    iter=10,)
model_nrc.save(f"model_nrc_2607.wordvectors")

INFO - 01:16:15: collecting all words and their counts
INFO - 01:16:15: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 01:16:17: PROGRESS: at sentence #10000, processed 5216036 words, keeping 493117 word types
INFO - 01:16:17: collected 604061 word types from a corpus of 6908852 raw words and 13212 sentences
INFO - 01:16:17: Loading a fresh vocabulary
INFO - 01:16:18: effective_min_count=5 retains 83933 unique words (13% of original 604061, drops 520128)
INFO - 01:16:18: effective_min_count=5 leaves 6223490 word corpus (90% of original 6908852, drops 685362)
INFO - 01:16:18: deleting the raw counts dictionary of 604061 items
INFO - 01:16:18: sample=0.001 downsamples 12 most-common words
INFO - 01:16:18: downsampling leaves estimated 6174794 word corpus (99.2% of prior 6223490)
INFO - 01:16:18: estimated required memory for 83933 words and 100 dimensions: 109112900 bytes
INFO - 01:16:18: resetting layer weights
INFO - 01:16:36: training model with 10 workers on

INFO - 01:17:16: EPOCH 5 - PROGRESS: at 25.54% examples, 697014 words/s, in_qsize 15, out_qsize 4
INFO - 01:17:17: EPOCH 5 - PROGRESS: at 37.28% examples, 703361 words/s, in_qsize 19, out_qsize 0
INFO - 01:17:18: EPOCH 5 - PROGRESS: at 47.37% examples, 695414 words/s, in_qsize 19, out_qsize 0
INFO - 01:17:19: EPOCH 5 - PROGRESS: at 58.09% examples, 695275 words/s, in_qsize 17, out_qsize 2
INFO - 01:17:20: EPOCH 5 - PROGRESS: at 69.10% examples, 691448 words/s, in_qsize 17, out_qsize 2
INFO - 01:17:21: EPOCH 5 - PROGRESS: at 80.62% examples, 695977 words/s, in_qsize 19, out_qsize 0
INFO - 01:17:22: EPOCH 5 - PROGRESS: at 90.87% examples, 690796 words/s, in_qsize 19, out_qsize 0
INFO - 01:17:23: worker thread finished; awaiting finish of 9 more threads
INFO - 01:17:23: worker thread finished; awaiting finish of 8 more threads
INFO - 01:17:23: worker thread finished; awaiting finish of 7 more threads
INFO - 01:17:23: worker thread finished; awaiting finish of 6 more threads
INFO - 01:17:2

INFO - 01:18:00: EPOCH 10 - PROGRESS: at 25.49% examples, 696456 words/s, in_qsize 16, out_qsize 3
INFO - 01:18:01: EPOCH 10 - PROGRESS: at 38.39% examples, 730365 words/s, in_qsize 19, out_qsize 0
INFO - 01:18:02: EPOCH 10 - PROGRESS: at 49.04% examples, 724740 words/s, in_qsize 20, out_qsize 1
INFO - 01:18:03: EPOCH 10 - PROGRESS: at 60.55% examples, 726908 words/s, in_qsize 17, out_qsize 2
INFO - 01:18:04: EPOCH 10 - PROGRESS: at 72.11% examples, 726533 words/s, in_qsize 19, out_qsize 0
INFO - 01:18:05: EPOCH 10 - PROGRESS: at 83.67% examples, 727007 words/s, in_qsize 18, out_qsize 1
INFO - 01:18:06: EPOCH 10 - PROGRESS: at 95.44% examples, 726737 words/s, in_qsize 19, out_qsize 0
INFO - 01:18:06: worker thread finished; awaiting finish of 9 more threads
INFO - 01:18:06: worker thread finished; awaiting finish of 8 more threads
INFO - 01:18:06: worker thread finished; awaiting finish of 7 more threads
INFO - 01:18:06: worker thread finished; awaiting finish of 6 more threads
INFO - 

In [58]:
#vocab len
for i in [model_40, model_50, model_60, model_70, model_80, model_90, model_nrc, model_tel, model_trouw, model_vk]:
    print(len(list(i.wv.vocab)))
    #print("unique tokens:",len(set(i.wv.vocab)))

9295
28294
40969
68516
123315
113668
83933
100169
72871
112943
